<a href="https://colab.research.google.com/github/NonaHelmi/SC/blob/main/antcolony_VRP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np

class Ant:
    def __init__(self, num_nodes):
        self.num_nodes = num_nodes
        self.route = []
        self.distance = 0

    def visit_city(self, city):
        if city not in self.route:
            self.route.append(city)

    def calculate_distance(self, distance_matrix):
        self.distance = 0
        for i in range(len(self.route) - 1):
            self.distance += distance_matrix[self.route[i]][self.route[i + 1]]
        self.distance += distance_matrix[self.route[-1]][self.route[0]]  # return to start

def aco_vrp(distance_matrix, num_vehicles, num_iterations):
    num_nodes = distance_matrix.shape[0]
    pheromone = np.ones((num_nodes, num_nodes)) / num_nodes
    best_route = None
    best_distance = float('inf')

    for _ in range(num_iterations):
        ants = [Ant(num_nodes) for _ in range(num_vehicles)]
        for ant in ants:
            start_city = np.random.randint(num_nodes)
            ant.visit_city(start_city)
            while len(ant.route) < num_nodes:
                next_city = np.random.choice(num_nodes, p=pheromone[start_city] / pheromone[start_city].sum())
                ant.visit_city(next_city)
                start_city = next_city
            ant.calculate_distance(distance_matrix)
            if ant.distance < best_distance:
                best_distance = ant.distance
                best_route = ant.route

        pheromone *= 0.95  # evaporation
        for ant in ants:
            for i in range(len(ant.route) - 1):
                pheromone[ant.route[i]][ant.route[i + 1]] += 1.0 / ant.distance

    return best_route, best_distance

# Example usage
distance_matrix = np.array([[0, 2, 9, 10],
                            [1, 0, 6, 4],
                            [15, 7, 0, 8],
                            [6, 3, 12, 0]])
num_vehicles = 2
num_iterations = 100
best_route, best_distance = aco_vrp(distance_matrix, num_vehicles, num_iterations)
print("Best route:", best_route)
print("Best distance:", best_distance)

Best route: [1, 0, 2, 3]
Best distance: 21
